In [1]:
from builtins import range
import numba
import numpy as np
import time
from pyem.util import rot2euler
from pyem.util import euler2rot
from pyem.util import euler2quat
from pyem.util import quat2euler
from pyem.util import quat2rot

In [69]:
@numba.jit
def cross(vec1, vec2):
    """ Calculate the cross product of two 3d vectors. """
    result = np.zeros(3, dtype=vec2.dtype)
    return cross_(vec1, vec2, result)


@numba.jit(nopython=True)
def cross_(vec1, vec2, result):
    """ Calculate the cross product of two 3d vectors. """
    a1, a2, a3 = vec1[0], vec1[1], vec1[2]
    b1, b2, b3 = vec2[0], vec2[1], vec2[2]
    result[0] = a2 * b3 - a3 * b2
    result[1] = a3 * b1 - a1 * b3
    result[2] = a1 * b2 - a2 * b1
    return result


@numba.jit(nopython=True)
def cross3(vec1, vec2, result):
    """ Calculate the cross product of two 3d vectors. """
    a1, a2, a3 = double(vec1[0]), double(vec1[1]), double(vec1[2])
    b1, b2, b3 = double(vec2[0]), double(vec2[1]), double(vec2[2])
    result[0] = a2 * b3 - a3 * b2
    result[1] = a3 * b1 - a1 * b3
    result[2] = a1 * b2 - a2 * b1

In [155]:
# @numba.jit(cache=True)
# def qconj(q, out=None):
#     if out is None:
#         out = q.copy()
#     if q.ndim == 1:
#         return _qconj(q, out)
#     else:
#         return _qconj_gu(q, out)


@numba.jit(cache=True, nopython=True)
def _qconj(q, p):
    p[0] = q[0]
    p[1] = -q[1]
    p[2] = -q[2]
    p[3] = -q[3]
    return p


@numba.guvectorize(["void(float64[:], float64[:])"],
                   "(m)->(m)",
                   nopython=True)
def qconj(q, p):
    _qconj(q, p)


# @numba.guvectorize(["void(float64[:,:], float64[:,:])"],
#                    "(m,n)->(m,n)",
#                    nopython=True)
# def _qconj_gu(q, p):
#     _qconj(q[0], p[0])


# @numba.jit(cache=True)
# def qtimes(q1, q2, out=None):
#     if out is None:
#         out = q1.copy()
#     if q1.ndim == 1:
#         return _qtimes(q1, q2, out)
#     else:
#         return _qtimes_gu(q1, q2, out)


@numba.jit(cache=True, nopython=True)
def _qtimes(q1, q2, q3):
    q3[0] = q1[0] * q2[0] - (q1[1] * q2[1] + q1[2] * q2[2] + q1[3] * q2[3])
    q3[1] = q1[2] * q2[3] - q1[3] * q2[2] + q1[0] * q2[1] + q2[0] * q1[1]
    q3[2] = q1[3] * q2[1] - q1[1] * q2[3] + q1[0] * q2[2] + q2[0] * q1[2]
    q3[3] = q1[1] * q2[2] - q1[2] * q2[1] + q1[0] * q2[3] + q2[0] * q1[3]
    return q3


# @numba.guvectorize(["void(float64[:,:], float64[:,:], float64[:,:])"],
#                    "(m,n),(m,n)->(m,n)",
#                    nopython=True)
# def _qtimes_gu(q1, q2, q3):
#     _qtimes(q1, q2, q3)


@numba.guvectorize(["void(float64[:], float64[:], float64[:])"],
                   "(m),(m)->(m)",
                   nopython=True)
def qtimes(q1, q2, q3):
    _qtimes(q1, q2, q3)


@numba.jit(cache=True, nopython=True)
def qslerp(q1, q2, t):
    cos_half_theta = np.dot(q1, q2)
    if cos_half_theta >= 1.0:
        return q1.copy()
    half_theta = np.arccos(cos_half_theta)
    sin_half_theta = np.sqrt(1 - cos_half_theta * cos_half_theta)
    if np.abs(sin_half_theta) < 1E-12:
        return (q1 + q2) / 2
    a = np.sin((1 - t) * half_theta) / sin_half_theta
    b = np.sin(t * half_theta) / sin_half_theta
    return q1 * a + q2 * b


# @numba.jit(cache=True, nopython=True)
# def qslerp(q1, q2, t):
#     q3 = np.zeros(t.shape[0], q1.shape[0], dtype=q1.dtype)
#     for i in t.shape[0]:
#         q3[i,:] = _qslerp(q1, q2, t[i])

In [38]:
qq = euler2quat(*np.deg2rad([45, 30, 30]))
pp = euler2quat(*np.deg2rad([0, 0, 90]))
qqr = qq.reshape(1, 4)
ppr = pp.reshape(1, 4)

rrr = np.vstack([qqr, qqr, qqr])
sss = np.vstack([ppr, ppr, ppr])

In [ ]:
out = qq.copy()
_qconj(qq, out)
%timeit _qconj_gu(qq, out)

In [28]:
out = qqr.copy()
%timeit _qconj_gu(qqr, out)

The slowest run took 43.73 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 687 ns per loop


In [37]:
out = qq.copy()
%timeit qconj(qq, out)
%timeit qconj(qq)
out = qqr.copy()
%timeit qconj(qqr, out)
%timeit qconj(qqr)

The slowest run took 52.44 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 609 ns per loop
The slowest run took 13.27 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 755 ns per loop
The slowest run took 12.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 645 ns per loop
The slowest run took 12.16 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 824 ns per loop


In [32]:
out = qq.copy()
%timeit _qtimes_gu(qq, pp, out)

The slowest run took 43.22 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 783 ns per loop


In [31]:
out = qqr.copy()
%timeit _qtimes_gu(qqr, ppr, out)

The slowest run took 34.15 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 761 ns per loop


In [33]:
out = qqr.copy()
%timeit _qtimes_gu(qqr, ppr)

The slowest run took 32.65 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 949 ns per loop


In [21]:
out = qq.copy()
%timeit qtimes(qq, pp, out)

The slowest run took 17.14 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.16 µs per loop


In [24]:
out = qqr.copy()
%timeit qtimes(qqr, ppr, out)

The slowest run took 25.25 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.46 µs per loop


In [35]:
%timeit qtimes(qq, pp)
%timeit qtimes(qqr, ppr)

The slowest run took 56.20 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 802 ns per loop
The slowest run took 13.24 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 846 ns per loop


In [41]:
%timeit qtimes(rrr, qq)
%timeit qtimes(rrr, sss)

The slowest run took 43.45 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 851 ns per loop
The slowest run took 11.45 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 875 ns per loop


In [148]:
qq = euler2quat(*np.deg2rad([0, 0, 0]))
pp = euler2quat(*np.deg2rad([0, 0, 90]))
print qq, np.linalg.norm(qq)
print pp, np.linalg.norm(pp)

[ 1.  0.  0.  0.] 1.0
[ 0.70710678  0.         -0.          0.70710678] 1.0


In [159]:
u = np.array([qslerp(qq, pp, t) for t in np.arange(0.1,1.1,0.1)])
print u
print[quat2euler(uu) for uu in u]

[[ 0.99691733  0.          0.          0.0784591 ]
 [ 0.98768834  0.          0.          0.15643447]
 [ 0.97236992  0.          0.          0.23344536]
 [ 0.95105652  0.          0.          0.30901699]
 [ 0.92387953  0.          0.          0.38268343]
 [ 0.89100652  0.          0.          0.4539905 ]
 [ 0.85264016  0.          0.          0.52249856]
 [ 0.80901699  0.          0.          0.58778525]
 [ 0.76040597  0.          0.          0.64944805]
 [ 0.70710678  0.          0.          0.70710678]]
[(0.0, nan, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, 0.0, 0.0), (0.0, nan, 0.0), (0.0, nan, 0.0), (0.0, 1.4901161193847656e-08, 0.0), (0.0, nan, 0.0), (0.0, nan, 0.0), (0.0, 0.0, 0.0)]


In [171]:
[np.rad2deg(rot2euler(quat2rot(uu))) for uu in u]

[array([ 0.,  0., -9.]),
 array([  0.,   0., -18.]),
 array([  0.,   0., -27.]),
 array([  0.,   0., -36.]),
 array([  0.,   0., -45.]),
 array([  0.,   0., -54.]),
 array([  0.,   0., -63.]),
 array([  0.,   0., -72.]),
 array([  0.,   0., -81.]),
 array([  0.,   0., -90.])]